Metadata
====
Information on the sex/age/etc. for our fish, and 

In [ ]:
import pathlib

parent_dir = pathlib.Path(
    "~/zebrafish_rdsf/Carran/Postgrad/Scale images from WT_spp1_sost/TIFs/segmentations"
).expanduser()
assert parent_dir.exists()

segmentation_paths = sorted([str(x) for x in parent_dir.glob("*.tif")])
f"{len(segmentation_paths)} segmentations"

In [ ]:
clean_seg_dir = parent_dir.parents[2] / "segmentations_cleaned"
clean_seg_paths = [clean_seg_dir / pathlib.Path(p).name for p in segmentation_paths]

# Check they all exist...
for p in clean_seg_paths:
    assert p.exists()

In [ ]:
import numpy as np

from scale_morphology.scales import metadata

df = metadata.df([str(x) for x in clean_seg_paths])
df.head()

In [ ]:
"""
Drop scales with missing data
"""

df = df[~df["no_scale"]]

In [ ]:
"""
Plot metadata
"""

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Replace NaN magnifications with -1
df.loc[np.isnan(df["magnification"]), "magnification"] = -1


def plot_mdata(df):
    vars = ["sex", "magnification", "age", "growth", "mutation"]
    plot_df = df[vars].copy()

    # So that we don't ignore NaN and inf
    plot_df = plot_df.sort_values(by="growth")
    plot_df["growth"] = plot_df["growth"].astype(str)

    # Sort them so they appear in a sensible order
    plot_df.loc[plot_df["growth"] == "inf", "growth"] = "onto"
    plot_df.loc[plot_df["growth"] == "nan", "growth"] = "?"

    sns.pairplot(plot_df, kind="hist", vars=vars)


plot_mdata(df)

Next, we'll find how many scales were manually edited (and how):

In [ ]:
"""
Read in pairs of the clean/raw segmentations, filling holes/removing small objects from the cleaned segmentation.

Keep track of which ones differ from the clean segmentation
"""

from concurrent.futures import ThreadPoolExecutor

import tifffile
import numpy as np
from tqdm.notebook import tqdm
from skimage.measure import euler_number
from scipy.ndimage import binary_fill_holes

from scale_morphology.scales.segmentation import largest_connected_component

scale_paths = df["path"]


def load_scale_data(paths):
    """
    Takes (i, (scale_path, raw_seg_path, clean_seg_path))

    Returns (cleaned_by_carran, fixed_in_post, raw_seg, carran_seg, clean_seg)
    """
    i, (scale_path, raw_seg_path, clean_seg_path) = paths

    raw_seg = (tifffile.imread(raw_seg_path) * 255).astype(np.uint8)
    carran_seg = tifffile.imread(clean_seg_path)

    cleaned_by_carran = (carran_seg != raw_seg).any()

    fixed_in_post = euler_number(carran_seg) != 1
    if fixed_in_post:
        # Fill holes
        clean_seg = binary_fill_holes(carran_seg)
        # Remove small objects
        clean_seg = (largest_connected_component(clean_seg) * 255).astype(np.uint8)

        # It's possible we might have removed everything, so just make sure we haven't here
        if euler_number(clean_seg) != 1:
            raise ValueError(f"Got {euler_number(clean_seg)=}")
    else:
        clean_seg = carran_seg

    return cleaned_by_carran, fixed_in_post, raw_seg, carran_seg, clean_seg


with ThreadPoolExecutor(max_workers=32) as executor:
    results = list(
        tqdm(
            executor.map(
                load_scale_data,
                enumerate(zip(scale_paths, segmentation_paths, clean_seg_paths)),
            ),
            total=len(scale_paths),
        )
    )

In [ ]:
raw_segs = []
carran_segs = []
clean_segs = []

# Which segmentations were edited by Carran and which segmentations we needed
# to fill holes/remove small objects from
cleaned_by_carran = []
fixed_in_post = []

for _carran_fix, _post_fix, _raw, _carran, _clean in results:
    cleaned_by_carran.append(_carran_fix)
    fixed_in_post.append(_post_fix)

    raw_segs.append(_raw)
    carran_segs.append(_carran)
    clean_segs.append(_clean)

cleaned_by_carran = np.array(cleaned_by_carran)
fixed_in_post = np.array(fixed_in_post)

raw_segs = np.array(raw_segs)
carran_segs = np.array(carran_segs)
clean_segs = np.array(clean_segs)

In [ ]:
"""
Show the masks that Carran edited
"""

import textwrap
import matplotlib.pyplot as plt
from matplotlib import colors


def clear_seismic() -> colors.Colormap:
    """
    Colormap that varies from blue to red via transparent
    """
    c_blue = colors.colorConverter.to_rgba("blue")
    c_white = colors.colorConverter.to_rgba("white", alpha=0)
    c_red = colors.colorConverter.to_rgba("red")
    return colors.ListedColormap([c_blue, c_white, c_red], f"clear2seismic")


fig, axes = plt.subplots(10, 10, figsize=(24, 24))

for axis, path, _raw, _cleaned in zip(
    axes.flat,
    df["path"][cleaned_by_carran],
    raw_segs[cleaned_by_carran],
    carran_segs[cleaned_by_carran],
):
    axis.imshow(_raw, alpha=0.5)
    axis.imshow(
        _raw.astype(int) - _cleaned,
        cmap=clear_seismic(),
        vmin=-255,
        vmax=255,
        interpolation="none",
    )

    axis.set_title(
        "\n".join(textwrap.wrap(pathlib.Path(path).name, width=20))
        .strip(".tif")
        .replace("_", " "),
        fontsize=8,
    )
    axis.set_axis_off()

fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(12, 12, figsize=(24, 24))

for axis, path, _raw, _cleaned in zip(
    axes.flat,
    df["path"][fixed_in_post],
    carran_segs[fixed_in_post],
    clean_segs[fixed_in_post],
):
    axis.imshow(_raw, alpha=0.5)

    diff = _raw.astype(int) - _cleaned

    y, x = np.argwhere(diff == -255).T
    axis.scatter(x, y, s=1, c="blue")

    y, x = np.argwhere(diff == 255).T
    axis.scatter(x, y, s=1, c="red")

    axis.set_title(
        "\n".join(textwrap.wrap(pathlib.Path(path).name, width=20))
        .strip(".tif")
        .replace("_", " "),
        fontsize=8,
    )
    axis.set_axis_off()

fig.tight_layout()